In [116]:
import os
import numpy as np
import pandas as pd

import sklearn.linear_model
import sklearn.tree
import sklearn.metrics
import sklearn.neighbors

from matplotlib import pyplot as plt
import pprint 
#import seaborn as sns


In [117]:
# determine whales to include based on what dataCSVs exist that dont exist

labeled_path = '../training_data/labeled_data/'
fv_path = '../training_data/fv_norm_CSVs/'
whale_names = list(os.walk(labeled_path))[0][2]
whale_names = [n[:-4] for n in whale_names]

CSV_names = [fv_path + n + '_images_fv_norm.csv' for n in whale_names]
labeled_names = [labeled_path + n + '.csv' for n in whale_names]


In [118]:
print(whale_names)
print(labeled_names)
print(CSV_names)
print(len(whale_names), len(labeled_names), len(CSV_names))

['Daffodil_1', 'Daffodil_2', 'Fan_1', 'Fan_2', 'Gom_1', 'Grommet_1']
['../training_data/labeled_data/Daffodil_1.csv', '../training_data/labeled_data/Daffodil_2.csv', '../training_data/labeled_data/Fan_1.csv', '../training_data/labeled_data/Fan_2.csv', '../training_data/labeled_data/Gom_1.csv', '../training_data/labeled_data/Grommet_1.csv']
['../training_data/fv_norm_CSVs/Daffodil_1_images_fv_norm.csv', '../training_data/fv_norm_CSVs/Daffodil_2_images_fv_norm.csv', '../training_data/fv_norm_CSVs/Fan_1_images_fv_norm.csv', '../training_data/fv_norm_CSVs/Fan_2_images_fv_norm.csv', '../training_data/fv_norm_CSVs/Gom_1_images_fv_norm.csv', '../training_data/fv_norm_CSVs/Grommet_1_images_fv_norm.csv']
6 6 6


In [119]:
x_data = []
y_data = []
for labeled_name, CSV_name, w in zip(labeled_names, CSV_names, whale_names):  
    xd = pd.read_csv(CSV_name, header = None)
    yd = pd.read_csv(labeled_name, header = None)
    print(w)
    x_data.append(xd)
    y_data.append(yd)
    

Daffodil_1
Daffodil_2
Fan_1
Fan_2
Gom_1
Grommet_1


In [121]:
print(y_data[0:2])
print(x_data[0:2])

[                                                     0   1
0    ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
1    ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
2    ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
3    ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
4    ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
..                                                 ...  ..
182  ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
183  ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
184  ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
185  ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N
186  ../whale_videos/Daffodil_1_images/Daffodil_1_0...   N

[187 rows x 2 columns],                                                     0   1
0   ../whale_videos/Daffodil_2_images/Daffodil_2_0...   N
1   ../whale_videos/Daffodil_2_images/Daffodil_2_0...   N
2   ../whale_videos/Daffodil_2_images/Daffodil_2_0...   N
3   ../whale_videos/Daffodil_2_ima

In [122]:
#y_data is an array of dataframes
#change this function to effect what the model trains to recognize
def process_y(arr):
    arr = [df.replace(to_replace='.*W.*', regex=True, value=1) for df in arr]
    arr = [df.replace(to_replace='.*N.*', regex=True, value=0) for df in arr]
    return(arr)
        
def y_trim(arr):
    arr = [df.drop(axis='columns', labels=0) for df in arr]
    return(arr)

#y_data = [process_y(arr) for arr in y_data]
y_data = process_y(y_data)
y_data = y_trim(y_data)


In [124]:
def process_x(arr):
    arr = [df.replace(to_replace='.*W.*', regex=True, value=1) for df in arr]
    arr = [df.replace(to_replace='.*N.*', regex=True, value=0) for df in arr]
    return(arr)
        
def x_trim(arr):
    arr = [df.drop(axis='columns', labels=0) for df in arr]
    return(arr)

x_data = x_trim(x_data)

In [125]:
print(x_data[0:2])

[            1         2         3         4         5         6
0    0.722568  0.694240  0.764017  0.488093  0.677637  0.869731
1    0.204116  0.191923  0.164480  0.605631  0.595384  0.586118
2    0.182167  0.159182  0.121784  0.435406  0.393424  0.403598
3    0.675768  0.618972  0.638899  0.417637  0.462871  0.710788
4    0.568055  0.500393  0.468637  0.343178  0.386711  0.548950
..        ...       ...       ...       ...       ...       ...
182  0.926927  0.910256  0.688156  0.119739  0.142415  0.482183
183  0.863290  0.841421  0.585722  0.086771  0.086559  0.321616
184  0.865841  0.847479  0.581066  0.105582  0.117456  0.369787
185  0.893437  0.878330  0.672460  0.117647  0.116134  0.418970
186  0.924324  0.910135  0.716196  0.124841  0.135924  0.454099

[187 rows x 6 columns],            1         2         3         4         5         6
0   0.812645  0.797989  0.567211  0.153418  0.239219  0.471961
1   0.799824  0.763983  0.552012  0.101493  0.111944  0.331504
2   0.783344  0.7

In [142]:
for i in range(len(whale_names)):
    x_icl = x_data[0:i] + x_data[i+1:]
    y_icl = y_data[0:i] + y_data[i+1:]
    x_tr = np.concatenate(x_icl)
    y_tr = np.concatenate(y_icl)
    x_test = x_data[i]
    y_test = y_data[i]
    #print(type(y_tr))
    print(x_tr.shape)
    print(y_tr.shape)
    print(y_tr[0:10])
    print(x_tr[0:10])
    clf = sklearn.linear_model.LogisticRegression(solver='liblinear').fit(x_tr, y_tr)
    #clf.score(x_test, y_test)
        
    
    
    

(340, 6)
(340, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]
[[0.81264487 0.7979889  0.56721087 0.15341847 0.23921945 0.47196136]
 [0.79982364 0.76398269 0.55201163 0.10149256 0.11194373 0.33150395]
 [0.78334445 0.75915142 0.52954428 0.12609661 0.15537848 0.35309019]
 [0.83965092 0.81114856 0.61933237 0.21055189 0.26450772 0.52349755]
 [0.73548454 0.69507562 0.53026741 0.1812403  0.20597058 0.35510784]
 [0.66744674 0.61082522 0.46341839 0.30654934 0.32947285 0.42832833]
 [0.77304805 0.74653826 0.55843836 0.25633157 0.30373569 0.49802321]
 [0.30959418 0.26839069 0.28281773 0.5008548  0.36031227 0.33807242]
 [0.57737213 0.52168067 0.44668144 0.21853279 0.17694777 0.28003086]
 [0.69064514 0.63639399 0.52076738 0.18897703 0.15698309 0.35593318]]
(459, 6)
(459, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]
[[0.72256808 0.69423958 0.76401736 0.48809262 0.6776368  0.86973058]
 [0.20411557 0.19192318 0.16447958 0.60563074 0.59538381 0.58611795]
 [0.18216739 0.1591816  0.1217840

c:\users\caleb\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\caleb\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\caleb\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\caleb\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversio

[1, 2]
